# Use a feature group to configure an AutoAI experiment

This notebook demonstrates how feature group metadata speeds up the configuration of an AutoAI experiment. The notebook describes the commands for retrieving a feature group, training experiments and scoring. The notebook uses the  project data asset called `german_credit_data_biased_training.csv`, which contains the German Credit Risk data set. This data asset has been enriched with feature group metadata. You can view the  _raw_ data set, without the metadata enrichment, at this location [german_credit_data_biased_training.csv](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/data/bias/german_credit_data_biased_training.csv).



## What you'll learn in this notebook

This notebook shows you how to:

- Retrieve existing feature group information for a data asset by using the `assetframe-lib` Python library.
- Quickly understand the training data using the feature group _preview_ method.
- Use feature group metadate to configure source and target columns in an AutoAI experiment, and to provide fairness information for AutoAI.


## Contents

This notebook contains the following parts:

1. [Before you start](#beforeYouStart)
2. [Work with feature groups](#featuregroup)
3. [Create an AutoAI experiment](#autoai)
4. [Summary](#summary)

<a id="beforeYouStart"></a>
## Before you start

### Set up Watson Machine Learning
To run the AutoAI experiment that is part of this notebook, you must ensure that Watson Machine Learning is installed in your CPD environment.

<a id="featuregroup"></a>
## Work with feature groups

### Overview

Feature groups are data assets that are useful for multiple machine learning use cases. Data engineers and data scientists can provide additional metadata for each feature of such a feature group that helps downstream machine learning tasks. This notebook uses two metadata attributes from the feature group: 
- The `role` of the feature: Should it be used as _input_ to a machine learning model, is it the _target_ of a prediction, or is it the _identifier_ of a partocular row of a data asset.
- The `fairness information`: this includes _monitored_ and _reference_ groups for input features, and the _favorable_ outcome for target features.

There are three options in Cloud Pak for Data to create and view feature groups. You can use:
- The _Feature group_ tab of data assets in the catalog UI
- The _Feature group_ tab of data assets in the project UI
- The _assetframe-lib_ Python library in notebooks

This notebook shows how to view feature groups using the `assetframe-lib` library.
For additional details on feature goups and the feature group tab, see [Managing data features](https://ibmdocs-test.dcs.ibm.com/docs/en/icpdaas_test?topic=data-managing-feature-groups-beta).

### Show the feature group and sample data

Begin by initializing the `assetframe-lib` library:

In [1]:
asset_name = "german_credit_data_biased_training.csv"

In [2]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

from assetframe_lib import AssetFrame

AssetFrame._wslib = wslib;

af = AssetFrame.from_data_asset(asset_name)

#### Data Preview
The `head()` method in the `assetframe-lib` library gives you a quick overview of the underlying data asset, without you having to import all the data into the notebook. Plus, it includes the following information about the data asset and the feature group:
- The _name_ of the data asset
- The _role_ of each feature
- The _description_ of each feature
- The _recipe_ of each feature
- For input features, it highlights values from the monitored groups (in yellow) and the reference groups (in brown)
- For target features, it highlights favorable outcomes (in green) and unfavorable outcomes (in red)


In [3]:
af.head()

#### Print all features

If you're only interested in the feature group metadata, use the name of the `assetframe` or `print(<assetframe name>)` in a notebook cell.

In [4]:
af

Role                                        Description  \
Age                  Input                                                      
EmploymentDuration   Input                                                      
Housing              Input                                                      
Job                  Input                                                      
LoanAmount           Input                                                      
LoanDuration         Input                                                      
LoanPurpose          Input                                                      
OthersOnLoan         Input  Whether there are other debtors or guarantors ...   
OwnsProperty         Input                                                      
Risk                Target                                                      
Sex                  Input                                                      

                   Favorable labels Unfavorable labels Monitored groups  \
Age                                                            [18, 25]   
EmploymentDuration                                                        
Housing                                                                   
Job                                                                       
LoanAmount                                                                
LoanDuration                                                              
LoanPurpose                                                               
OthersOnLoan                                                              
OwnsProperty                                                              
Risk                        No Risk               Risk                    
Sex                                                              female   

                   Reference groups  \
Age                        [26, 75]   
EmploymentDuration                    
Housing                               
Job                                   
LoanAmount                            
LoanDuration                          
LoanPurpose                           
OthersOnLoan                          
OwnsProperty                          
Risk                                  
Sex                            male   

                                                   Value descriptions Recipe  \
Age                                                                            
EmploymentDuration                                                             
Housing                                                                        
Job                                                                            
LoanAmount                                                                     
LoanDuration                                                                   
LoanPurpose                                                                    
OthersOnLoan        ('none', 'No others on the loan'), ('co-applic...          
OwnsProperty                                                                   
Risk                                                                           
Sex                                                                            

                   Tags  
Age                      
EmploymentDuration       
Housing                  
Job                      
LoanAmount               
LoanDuration             
LoanPurpose              
OthersOnLoan             
OwnsProperty             
Risk                     
Sex

### Collect the metadata for the AutoAI experiment

To start an AutoAI experiment with a data asset, you need to provide:
- The columns to be used as input to the model
- The column to be used as target
- Fairness information

With `assetframe-lib` methods, you can easily access this specific metadata, and retrieve it in a form that can be used directly in AutoAI. Here's how:

#### Get the input features

In [5]:
input_features = af.get_features_by_role("input")

In [6]:
input_feature_names = list(map(lambda feature: feature.get_column_name(), input_features))
print(input_feature_names)

['Age', 'EmploymentDuration', 'Housing', 'Job', 'LoanAmount', 'LoanDuration', 'LoanPurpose', 'OthersOnLoan', 'OwnsProperty', 'Sex']


#### Get the target feature

In the example, only one feature is labeled as `target`, which is why you can access the result directly using `[0]`.

In [7]:
target_features = af.get_features_by_role("target")

In [8]:
target_feature_name = target_features[0].get_column_name()
print(target_feature_name)

Risk


#### Get the fairness information for input and target features

You could get the information for each feature separately, using the following pattern:
```
feature = af.get_feature(<name>)
monitoredGroups = feature.get_monitored_groups()
referenceGroups = feature.get_reference_groups()
``` 

However, the `assetframe-lib` library provides a convenient method called `get_fairness_info()` that you can use to retrieve _all_ fairness information for a feature group, in a format that can be directly used in AutoAI, or other libraries such as [AI Fairness 360](https://aif360.mybluemix.net/resources#overview). 

In [9]:
fairness_info = af.get_fairness_info()

In [10]:
fairness_info

{'favorable_labels': ['No Risk'],
 'unfavorable_labels': ['Risk'],
 'protected_attributes': [{'feature': 'Age',
   'monitored_group': [[18, 25]],
   'reference_group': [[26, 75]]},
  {'feature': 'Sex',
   'monitored_group': ['female'],
   'reference_group': ['male']}]}

<a id="autoai"></a>
## Create an AutoAI experiment

Now, you will learn to use AutoAI to build a model that predicts `Risk` (our target feature), given the input features from the feature group.

### Connect to the IBM Watson Machine Learning service

In [25]:
url = '<YOUR CPD PLATFORM URL>'

In [37]:
access_token = wslib.auth.get_current_token()

wml_credentials = {
    "instance_id": "openshift",
    "token": access_token,
    "url": url, 
    "version" : "4.7"
}

In [38]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Connect to the credit risk data

In [39]:
from ibm_watson_machine_learning.helpers import DataConnection, AssetLocation

asset_details = wslib.assets.get_stored_data(asset_name, raw=True)
asset_id = asset_details["metadata"]["asset_id"]

credit_risk_conn = DataConnection(data_asset_id=asset_id)
training_data_reference=[credit_risk_conn]

### Configure the experiment with the feature group metadata

Now, you can create the AutoAI experiment. You will use the feature group metadata to configure key aspects of the experiment:
- `prediction_column` is the `target_feature_name`
- `fairness_info` is the information from `get_fairness_info()`
- `train_sample_columns_list`is the list of `input_feature_names`

In [40]:
from ibm_watson_machine_learning.experiment import AutoAI

project_id = wslib.here.get_ID()

experiment = AutoAI(wml_credentials, project_id)

pipeline_optimizer = experiment.optimizer(
    name='Credit Risk Prediction - AutoAI',
    desc='Credit Risk Model using Feature Group',
    prediction_type=AutoAI.PredictionType.BINARY,
    prediction_column=target_feature_name,
    fairness_info=fairness_info,
    train_sample_columns_list=input_feature_names,
    scoring=AutoAI.Metrics.ROC_AUC_SCORE,
)

`get_params()` shows that the feature group metadata was retrieved:

In [41]:
pipeline_optimizer.get_params()['fairness_info']

{'favorable_labels': ['No Risk'],
 'unfavorable_labels': ['Risk'],
 'protected_attributes': [{'feature': 'Age',
   'monitored_group': [[18, 25]],
   'reference_group': [[26, 75]]},
  {'feature': 'Sex',
   'monitored_group': ['female'],
   'reference_group': ['male']}]}

### Run the experiment

You can now run the experiment by calling the `fit()` method. `backgroundMode=False` ensures that the experiment is complete when the notebook cell is completed. 

In [42]:
run_details = pipeline_optimizer.fit(
            training_data_reference=training_data_reference,
            background_mode=False)

Training job ed53187f-c645-434e-9f9d-8635ffab2755 completed: 100%|████████| [02:33<00:00,  1.54s/it]


### Pipelines comparison

You can list trained pipelines and evaluation metrics information in
the form of a pandas DataFrame by calling the `summary()` method. You can
use the DataFrame to compare all discovered pipelines and select the one
you like for further testing.

Notice the columns for `training_disparate_impact_Sex` and `training_disparate_impact_Age`. AutoAI used the fairness information for both columns to compute these fairness metrics. 

In [43]:
summary = pipeline_optimizer.summary()
summary

Enhancements  \
Pipeline Name                           
Pipeline_7                        HPO   
Pipeline_1                              
Pipeline_3                    HPO, FE   
Pipeline_4               HPO, FE, HPO   
Pipeline_5     HPO, FE, HPO, Ensemble   
Pipeline_8                    HPO, FE   
Pipeline_9               HPO, FE, HPO   
Pipeline_10    HPO, FE, HPO, Ensemble   
Pipeline_2                        HPO   
Pipeline_6                              

                                                       Estimator  \
Pipeline Name                                                      
Pipeline_7                                         XGBClassifier   
Pipeline_1                         SnapBoostingMachineClassifier   
Pipeline_3                         SnapBoostingMachineClassifier   
Pipeline_4                         SnapBoostingMachineClassifier   
Pipeline_5     BatchedTreeEnsembleClassifier(SnapBoostingMach...   
Pipeline_8                                         XGBClassifier   
Pipeline_9                                         XGBClassifier   
Pipeline_10         BatchedTreeEnsembleClassifier(XGBClassifier)   
Pipeline_2                         SnapBoostingMachineClassifier   
Pipeline_6                                         XGBClassifier   

               training_roc_auc_(optimized)  training_disparate_impact_Sex  \
Pipeline Name                                                                
Pipeline_7                         0.838453                       0.995341   
Pipeline_1                         0.834072                       1.141928   
Pipeline_3                         0.837266                       1.141034   
Pipeline_4                         0.837266                       1.141034   
Pipeline_5                         0.837266                       1.141034   
Pipeline_8                         0.840450                       0.981082   
Pipeline_9                         0.840450                       0.981082   
Pipeline_10                        0.840450                       0.981082   
Pipeline_2                         0.835565                       1.186798   
Pipeline_6                         0.831296                       0.988735   

               training_disparate_impact  holdout_disparate_impact_Sex  \
Pipeline Name                                                            
Pipeline_7                      2.112797                      0.914803   
Pipeline_1                      2.613027                      1.078119   
Pipeline_3                      3.246667                      1.048070   
Pipeline_4                      3.246667                      1.048070   
Pipeline_5                      3.246667                      1.048070   
Pipeline_8                      1.963754                      0.925435   
Pipeline_9                      1.963754                      0.925435   
Pipeline_10                     1.963754                      0.925435   
Pipeline_2                      4.143579                      1.053409   
Pipeline_6                      1.818891                      0.948684   

               holdout_average_precision  holdout_log_loss  holdout_roc_auc  \
Pipeline Name                                                                 
Pipeline_7                      0.476960          0.412361         0.849019   
Pipeline_1                      0.468686          0.408897         0.848838   
Pipeline_3                      0.478866          0.458079         0.846034   
Pipeline_4                      0.478866          0.458079         0.846034   
Pipeline_5                      0.478866          0.458079         0.846034   
Pipeline_8                      0.474951          0.404046         0.845520   
Pipeline_9                      0.474951          0.404046         0.845520   
Pipeline_10                     0.474951          0.404046         0.845520   
Pipeline_2                      0.478234          0.456256         0.845015   
Pipeline_6          

Looking at `Pipeline_1` , you see that AutoAI only used the `Input` features for the model. 

In [44]:
pipeline_details = pipeline_optimizer.get_pipeline_details('Pipeline_1')

In [45]:
pipeline_details['features_importance']

features_importance
Age                                1.00
LoanDuration                       0.77
LoanAmount                         0.52
EmploymentDuration                 0.37
OwnsProperty                       0.34
OthersOnLoan                       0.13
Sex                                0.10
Housing                            0.01
LoanPurpose                        0.00
Job                                0.00

<a id="summary"></a>
## Summary

Congratulations! You retrieved an existing feature group, and used its metadata to configure an AutoAI experiment. Feel free to extend this notebook to pick one of the AutoAI pipelines, and deploy the model.

### Authors

**Szymon Brandys**, Senior Software Engineer in CloudPak for Data, IBM

**Simone Zerfass**, Software Developer, Watson Studio, IBM

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.